In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import sys
import os

import sklearn
from sklearn import cross_validation
from sklearn import svm
from sklearn import metrics

reload(sys)
#sys.setdefaultencoding("ISO-8859-1")
sys.setdefaultencoding("UTF-8")

def clean_text(raw_text):
    review_text = BeautifulSoup(raw_text).get_text()
    words = review_text.lower().split()
    return(" ".join(words))

In [2]:
languages = ["english"]
datafiles = ["summary-english-truth.txt"]
tasks = ["age"]
for language, datafile in zip(languages, datafiles):
    train = pd.read_csv(datafile, header=0, delimiter="\t", quoting=1)
    num_text = train["text"].size
    clean_train_data = []
    
    for i in xrange( 0, num_text):
        clean_train_data.append( clean_text( train["text"][i] ) )
    vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None)
    train_x = vectorizer.fit_transform(clean_train_data)
    train_x = train_x.toarray()
    print "shape: ", train_x.shape
    rows, cols = train_x.shape

In [3]:
with open('age-important-words-using-info-gain.txt') as f:
    alist = [line.rstrip() for line in f]
all_indices_ranked = alist[0].split(',')
all_indices_ranked = [int(x) for x in all_indices_ranked]
all_indices_ranked = [x-1 for x in all_indices_ranked]

In [4]:
num_features = 9000
xx = [all_indices_ranked[x] for x in range(0, num_features)]
xx = tuple(xx)
age_features_smaller_train_x = train_x[:, xx]

In [5]:
with open('gender-important-words-info-gain.txt') as f:
    alist = [line.rstrip() for line in f]
all_indices_ranked = alist[0].split(',')
all_indices_ranked = [int(x) for x in all_indices_ranked]
all_indices_ranked = [x-1 for x in all_indices_ranked]

In [6]:
num_features = 7000
xx = [all_indices_ranked[x] for x in range(0, num_features)]
xx = tuple(xx)
gender_features_smaller_train_x = train_x[:, xx]

In [7]:
train = pd.read_csv("summary-english-truth.txt", header=0, delimiter="\t", quoting=1)
train_y_age = train['age']
list_of_scores = []

In [8]:
train = pd.read_csv("summary-english-truth.txt", header=0, delimiter="\t", quoting=1)
train_y_gender = train['gender']
list_of_scores = []

In [9]:
clf1 = svm.SVC(kernel='poly', degree=3, coef0=10, gamma=1)

In [10]:
scores_age_with_gender_features = cross_validation.cross_val_score(clf1, gender_features_smaller_train_x, train_y_age, cv=10, scoring='accuracy')

In [11]:
scores_age_with_gender_features.mean()

0.7333613445378151

In [12]:
clf2 = svm.SVC(kernel='poly', degree=2, coef0=10000, gamma=1)

In [13]:
scores_gender_with_age_features = cross_validation.cross_val_score(clf2, age_features_smaller_train_x, train_y_gender, cv=10, scoring='accuracy')

In [14]:
scores_gender_with_age_features.mean()

0.74375000000000013

In [21]:
clf3 = svm.SVC(kernel='poly', degree=3, coef0=10, gamma=1)
scores_age_with_age_features = cross_validation.cross_val_score(clf3, age_features_smaller_train_x, train_y_age, cv=10, scoring='accuracy')

In [22]:
scores_age_with_age_features.mean()

0.80960784313725487

In [23]:
clf4 = svm.SVC(kernel='poly', degree=2, coef0=10000, gamma=1)
scores_gender_with_gender_features = cross_validation.cross_val_score(clf4, gender_features_smaller_train_x, train_y_gender, cv=10, scoring='accuracy')

In [24]:
scores_gender_with_gender_features.mean()

0.78035714285714275

In [25]:
#testing statistical significance

In [26]:
from scipy import stats
import pandas as pd

z_stat, p_val_age = stats.ranksums(scores_age_with_gender_features, scores_age_with_age_features)


In [27]:
p_val_age

0.15092695006671628

In [29]:
z_stat, p_val_gender = stats.ranksums(scores_gender_with_age_features, scores_gender_with_gender_features)

In [30]:
p_val_gender

0.52052288327577267

-0.64253960411568634